In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
##library used
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium
import plotly.express as px

pd.set_option("display.max_columns", 36)
sns.set(style="whitegrid")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/hotel-booking-demand/hotel_bookings.csv")
df.head(2)

Predicting whether a person booking hotel will cancel their booking or no.

In [ ]:
sns.barplot(x="is_canceled", y="arrival_date_month", data=df)

In [ ]:
sns.barplot(y="is_canceled", x='hotel', data=df)

In [ ]:
sns.barplot(y="is_canceled", x="deposit_type", data=df)

In [ ]:
char = df.select_dtypes(include='object')
for i in char:
    print("----------------")
    print(df[i].value_counts())
    

In [ ]:
df.isnull().sum()

In [ ]:
df['agent'] = df['agent'].fillna(0)
df['children'] = df['children'].fillna(0)
df = df.drop(['company','country'], axis = 1)

In [ ]:
df.head(2)

In [ ]:
char = df.select_dtypes(include='object')
for i in char:
    print(i , df[i].nunique())

In [ ]:
df.sample(2) 
#arrival_date_month

In [ ]:

df['arrival_date_year'] = pd.to_datetime(df['arrival_date_year'])
df['arrival_date_year'] = df['arrival_date_year'].dt.year

df['arrival_date_day_of_month'] = pd.to_datetime(df['arrival_date_day_of_month'])
df['arrival_date_day_of_month'] = df['arrival_date_day_of_month'].dt.month


df['arrival_date_week_number'] = pd.to_datetime(df['arrival_date_week_number'])
df['arrival_date_week_number'] = df['arrival_date_week_number'].dt.day

df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])
df['reservation_day'] = df['reservation_status_date'].dt.day
df['reservation_month'] = df['reservation_status_date'].dt.month
df['reservation_year'] = df['reservation_status_date'].dt.year
df.drop(['reservation_status_date','arrival_date_month'] , axis = 1, inplace = True)

In [ ]:
df.dtypes

In [ ]:
a = df.select_dtypes(object).columns
cat_list = []
for i in a:
    print (i, df[i].nunique())
    cat_list.append(i)

In [ ]:
df.head(2)

In [ ]:
useless_col = ['days_in_waiting_list', 'arrival_date_year', 'arrival_date_year', 'assigned_room_type', 'booking_changes',
               'reservation_status', 'days_in_waiting_list']
df.drop(useless_col, axis = 1, inplace = True)

In [ ]:
##sns.pairplot(df, hue="is_canceled", diag_kws={"hue": None, "color": ".2"})

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in df:
    df[col] = le.fit_transform(df[col].astype(str).values)

In [ ]:
from sklearn.model_selection import train_test_split 
X = df.drop("is_canceled", axis=1)
y = df['is_canceled']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=101,test_size=0.3)
X_train

In [ ]:
##from sklearn.preprocessing import StandardScaler
##scaler = StandardScaler()
##x_train = scaler.fit_transform(X_train)
##x_test = scaler.transform(X_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
cart = DecisionTreeClassifier(max_depth = 12)
cart_model = cart.fit(X_train, y_train)
y_pred_dtc = cart_model.predict(X_test)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
acc_dtc = accuracy_score(y_test, y_pred_dtc)
conf = confusion_matrix(y_test, y_pred_dtc)
clf_report = classification_report(y_test, y_pred_dtc)
print(acc_dtc)
print(conf)
print(clf_report)

In [ ]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(solver='liblinear')
logreg = log.fit(X_train,y_train)
y_pred_lr = logreg.predict(X_test)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
acc_lr = accuracy_score(y_test, y_pred_lr)
conf_lr = confusion_matrix(y_test, y_pred_lr)
clf_report_lr = classification_report(y_test, y_pred_lr)
print(acc_lr)
print(conf_lr)
print(clf_report_lr)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf1 = rf.fit(X_train, y_train)
y_pred_rf = rf1.predict(X_test)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
acc_rf = accuracy_score(y_test, y_pred_rf)
conf_rf = confusion_matrix(y_test, y_pred_rf)
clf_report_rf = classification_report(y_test, y_pred_rf)
print(acc_rf)
print(conf_rf)
print(clf_report_rf)

In [ ]:
from xgboost import XGBClassifier
xg = XGBClassifier()
xg = rf.fit(X_train, y_train)
y_pred_xg = rf1.predict(X_test)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
acc_xg = accuracy_score(y_test, y_pred_xg)
conf_xg = confusion_matrix(y_test, y_pred_xg)
clf_report_xg = classification_report(y_test, y_pred_xg)
print(acc_xg)
print(conf_xg)
print(clf_report_xg)

In [ ]:
##from sklearn.svm import SVC
##clf = SVC()
##clf1 = clf.fit(X_train, y_train)
##y_pred_svc = clf1.predict(X_test)

In [ ]:
from sklearn.metrics import roc_curve, confusion_matrix, auc
def ROC(y_test, y_prob):
    
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test, y_prob)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    
    plt.figure(figsize = (10,10))
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positive_rate, true_positive_rate, color = 'red', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1], linestyle = '--')
    plt.axis('tight')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')

In [ ]:
print("XGBClassifier")
ROC(y_test, y_pred_xg)